In [26]:
import open3d as o3d
import numpy as np
import copy

view_count = 16

def ICP_matrix(source, target, iterations=10, threshold=.02, trans_init=np.identity(4)):
    source_copy = copy.deepcopy(source)

    cumulative_transformation = np.identity(4)

    for _ in range(iterations):
        source_filtered, _ = source_copy.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
        target_filtered, _ = target.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
        source_downsampled = source_filtered.voxel_down_sample(voxel_size=0.001)
        target_downsampled = target_filtered.voxel_down_sample(voxel_size=0.001)

        reg_p2p = o3d.pipelines.registration.registration_icp(
            source_downsampled, target_downsampled, threshold, trans_init,
            o3d.pipelines.registration.TransformationEstimationPointToPoint())
        cumulative_transformation = np.dot(reg_p2p.transformation, cumulative_transformation)
        source_copy.transform(reg_p2p.transformation)

    return cumulative_transformation

def mstorePCR(point_clouds, smoothing=False, verbose=0):
    result = point_clouds[0]
    
    for i in range(len(point_clouds)-1):
        source, target = point_clouds[i+1], point_clouds[i]
        m = ICP_matrix(source, target)

        source.transform(m)
        result += source

        if smoothing:
            result, _ = result.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
            result = result.voxel_down_sample(voxel_size=0.001)

    if verbose:
        o3d.visualization.draw_geometries([result])
    return result

In [27]:
from visual import standardICP, piece_pci

def piece_pci_overlap(pcs):
    while len(pcs) > 1:
        print("Starting loop...")
        # Piece Creation
        new_pcs = []
        for i in range(0, len(pcs), 3):
            new_pcs.append(standardICP(pcs[i] + pcs[i-1], pcs[i-1] + pcs[i-2]))
            if i <= len(pcs) - 3:
                new_pcs.append(standardICP(pcs[i] + pcs[i+1], pcs[i+1] + pcs[i+2]))
            elif i > len(pcs) - 3:
                new_pcs.append(standardICP(pcs[i],  pcs[i+1]))

        pcs = new_pcs

        # Piece Joining
        new_pcs = []
        for i in range(0, len(pcs), 2):
            new_pcs.append(standardICP(pcs[i] , pcs[i+1]))

        pcs = new_pcs
        print("Ending loop...")
    return pcs[0]

In [28]:
import robotic as ry
import json

point_cloud_name = "blocks"
pcs = [o3d.io.read_point_cloud(f"./data/pcs/{point_cloud_name}/point_cloud_{i}.pcd") for i in range(view_count)]
result = mstorePCR(pcs, verbose=0)
#result_old = piece_pci(pcs)

In [29]:
ry.params_print()
verbose = 0

C = ry.Config()

for i in range(view_count):
    v = json.load(open(f"./data/pcs/{point_cloud_name}/point_cloud_{i}.json"))
    pclFrame = C.addFrame(f'pcl{i}')
    pclFrame.setPointCloud(np.array(v))
    pclFrame.setColor([0.,1.,0.])
    pclFrame.setPosition([0,.30,0])
    C.view_recopyMeshes()

final_points = np.asarray(result.points)
pclFrame = C.addFrame('pcl')
pclFrame.setPointCloud(np.array(final_points))
pclFrame.setColor([0.,0.,1.])
C.view_recopyMeshes()

#final_points_old = np.asarray(result_old.points)
#pclFrame = C.addFrame('pcl_old')
#pclFrame.setPointCloud(result_old)
#pclFrame.setColor([0.,0.,1.])
#pclFrame.setPosition([0,-.30,0])
#C.view_recopyMeshes()

C.view(True)

-- module.cpp:operator():95(0) python,
RealSense/resolution: 640,
RealSense/alignToDepth!,
physx/motorKp: 10000,
physx/motorKd: 1000,
physx/multiBody!


0